# Standart PA

https://jira.x5.ru/browse/CVMUPG-1141

In [ ]:
name = 'CVMXC-2801' + '_' + 'danone' + '_pa'

In [ ]:
import sys
import pandas as pd
import pyspark.sql.functions as F
sys.path.append('/home/jovyan/cvm_upgrade/')
sys.path.append('/home/jovyan/cvm_upgrade/upgrade/')
sys.path.append('/home/jovyan/glow-byte-filters-pyspark')
sys.path.append('/home/jovyan/x5_some_tasks/Osipov/')
from upgrade.post_analysis2.tools.groups_metrics_aggregation import GroupsMetricsAggregation
from upgrade.post_analysis2.tools.guests_metrics_preprocessing import GuestsMetricsPreprocessing
from upgrade.post_analysis2.tools.post_analysis_params import PostAnalysisParams
from upgrade.post_analysis2.tools.metric_names_mapper import map_col_names_to_russian
from upgrade.post_analysis2.tools.experiment_participants import ExperimentParticipants
from upgrade.post_analysis2.tools.experiment_params import ExperimentParams
import datetime
from datetime import timedelta
from spark import *
from utils_osipov import *

%load_ext autoreload
%autoreload 2

pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_rows = 500

In [ ]:
sc, spark = restart_spark(name, 100, executor_memory="3G", executor_cores=5, driver_memory="12G",
                          additional_params={"spark.sql.shuffle.partitions": "200"})
sc.setLogLevel('ERROR')

### 1: X-28-42-Mareven-BigBon-sof

In [ ]:
campaign_id = 'X-28-42-Mareven-BigBon-sof'
output_fin_report = f'{campaign_id}_fin_report.csv'
output_main_report = f'{campaign_id}_main_report.csv'

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        80782: "sms_viber_tg",
        80782: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': '2022-01-25',
    'date_communication_to': '2022-01-31',

    'wave_ids': [
        81240 #если нет, то -1
    ],

    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})

post_analysis_params = PostAnalysisParams(**{
    'spark': spark,
    'date_from': '2022-01-25',
    'date_to': '2022-01-31', #дата постпериода
    'control_camp_wave_id': 80782,
    'plu_list': list(map(str, [3003111,2082200,4025330,4059568,4063579,4025331,2082204,4025363,4115113,
                               3368169,3003110,2082206,4113988,3368168,3468665,3468666,4115114,4113990])),
    'threshold_item_condition': 'plu_list', #есть категория еще
    'count_for_plu_list': 2 #механика от 2-х штук
})


assert post_analysis_params.control_camp_wave_id in experiment_params.map_camp_wave_ids_to_suffixes.keys()

In [ ]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [ ]:
guest_metrics_preparation = GuestsMetricsPreprocessing(spark, experiment_guests_communications, post_analysis_params)
experiment_guests_with_metrics = guest_metrics_preparation.evaluate()

In [ ]:
groups_metrics_aggregation = GroupsMetricsAggregation(spark, experiment_guests_with_metrics, post_analysis_params)
groups_metrics, map_to_effects_table = groups_metrics_aggregation.evaluate()

In [ ]:
fin_report = map_col_names_to_russian(groups_metrics)
fin_report

In [ ]:
fin_report['meta_post_analysis'] = str(post_analysis_params.__dict__)
fin_report['meta_experiment_params'] = str(experiment_params.__dict__)
groups_metrics['meta_post_analysis'] = str(post_analysis_params.__dict__)
groups_metrics['meta_experiment_params'] = str(experiment_params.__dict__)

groups_metrics.T.to_csv(output_main_report, encoding='utf-8-sig', sep=';')
fin_report.T.to_csv(output_fin_report, encoding='utf-8-sig', sep=';')